In [1]:
#importation des bibliothèques

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as plt
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
from IPython import display
from datetime import datetime
from datetime import date
import matplotlib.pyplot as plt
import warnings
plt.style.use('ggplot')
pd.set_option('display.max_columns', None)
warnings.filterwarnings("ignore")

In [2]:
LABEVENTS='./data/LABEVENTS.csv'
PRESCRIP='./data/PRESCRIPTIONS.csv'
ADMIS ='./data/ADMISSIONS.csv'
PROCED='./data/D_ICD_PROCEDURES.csv'
PROCED_ICD='./data/PROCEDURES_ICD.csv'
PAT='./data/PATIENTS.csv'
DIAGNO='./data/D_ICD_DIAGNOSES.csv'
DIAGNO_ICD='./data/DIAGNOSES_ICD.csv'
NOTEEVENTS='./data/NOTEEVENTS.csv'


In [3]:
admissions_df=pd.read_csv('./data/ADMISSIONS.csv')

In [4]:
# Supprimer les lignes contenant des valeurs nulles
admissions_df.dropna(axis=0, inplace=True)

In [5]:
d_icd_proc_df=pd.read_csv('./data/D_ICD_PROCEDURES.csv')
# Supprimer les lignes contenant des valeurs nulles
d_icd_proc_df.dropna(axis=0, inplace=True)
d_icd_proc_df.shape

(3882, 4)

In [6]:
procedures_icd_df=pd.read_csv('./data/PROCEDURES_ICD.csv')
# Supprimer les lignes contenant des valeurs nulles
procedures_icd_df.dropna(axis=0, inplace=True)
procedures_icd_df.shape

(240095, 5)

In [7]:
procedures_icd_df.head(100)

,ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE
0,944,62641,154460,3,3404
1,945,2592,130856,1,9671
2,946,2592,130856,2,3893
3,947,55357,119355,1,9672
4,948,55357,119355,2,331
...,...,...,...,...,...
95,1039,24730,143418,1,370
96,1040,24730,143418,2,3491
97,1041,6466,185829,1,8411
98,1042,6466,185829,2,3929


In [8]:
# Fusionner les DataFrames'd_icd_proc_df' et 'procedures_icd_dff' sur la colonne SUBJECT_ID
merged_procedure = pd.merge(d_icd_proc_df, procedures_icd_df[['ICD9_CODE','SUBJECT_ID','HADM_ID','SEQ_NUM']], on='ICD9_CODE', how='inner')

In [17]:
merged_procedure.head()


,ROW_ID,ICD9_CODE,SHORT_TITLE,LONG_TITLE,SUBJECT_ID,HADM_ID,SEQ_NUM
0,264,851,Canthotomy,Canthotomy,16637,133896,1
1,264,851,Canthotomy,Canthotomy,18875,189045,1
2,264,851,Canthotomy,Canthotomy,13516,191600,1
3,264,851,Canthotomy,Canthotomy,23407,171982,1
4,264,851,Canthotomy,Canthotomy,13376,166959,9


In [21]:
merged_procedure.shape

(246178, 7)

In [19]:
merged_diagnos = merged_procedure.drop_duplicates(['SUBJECT_ID', 'HADM_ID'])


In [22]:
# Liste des noms des colonnes à supprimer
colonnes_a_supprimer = ['SHORT_TITLE','ROW_ID']
#Supprimer plusieurs colonnes simultanément
merged_procedure = merged_procedure.drop(columns=colonnes_a_supprimer)
merged_procedure.head()

,ICD9_CODE,LONG_TITLE,SUBJECT_ID,HADM_ID,SEQ_NUM
0,851,Canthotomy,16637,133896,1
1,851,Canthotomy,18875,189045,1
2,851,Canthotomy,13516,191600,1
3,851,Canthotomy,23407,171982,1
4,851,Canthotomy,13376,166959,9


In [23]:
copie_admissions_df=pd.DataFrame(admissions_df)

In [24]:
# Liste des noms des colonnes à supprimer
colonnes_a_supprimer = ['ROW_ID','EDREGTIME','EDOUTTIME']
#Supprimer plusieurs colonnes simultanément
copie_admissions_df = copie_admissions_df.drop(columns=colonnes_a_supprimer)


In [13]:
# Fusionner les DataFrames'copie_admissions_df' et 'merged_procedure' sur la colonne SUBJECT_ID
merged_procedure = pd.merge(copie_admissions_df, merged_procedure, on=['SUBJECT_ID', 'HADM_ID'], how='inner')

In [25]:
merged_procedure.shape

(246178, 5)

In [28]:
patients_df=pd.read_csv('./data/PATIENTS.csv')

In [30]:
patients_df.shape

(46520, 8)

In [31]:
# Supprimer les lignes contenant des valeurs nulles
patients_df.shape

(46520, 8)

In [32]:
patients_df.head()

,ROW_ID,SUBJECT_ID,GENDER,DOB,DOD,DOD_HOSP,DOD_SSN,EXPIRE_FLAG
0,234,249,F,2075-03-13 00:00:00,NaN,NaN,NaN,0
1,235,250,F,2164-12-27 00:00:00,2188-11-22 00:00:00,2188-11-22 00:00:00,NaN,1
2,236,251,M,2090-03-15 00:00:00,NaN,NaN,NaN,0
3,237,252,M,2078-03-06 00:00:00,NaN,NaN,NaN,0
4,238,253,F,2089-11-26 00:00:00,NaN,NaN,NaN,0


In [33]:
# Fusionner les DataFrames'patients_df' et 'Mimic_III_df' sur la colonne SUBJECT_ID
Mimi_III_df = pd.merge(merged_procedure, patients_df[['SUBJECT_ID', 'GENDER', 'DOB']], on='SUBJECT_ID', how='inner')

In [40]:
Mimi_III_df.head()

,ICD9_CODE,LONG_TITLE,SUBJECT_ID,HADM_ID,SEQ_NUM,GENDER,DOB,age
0,851,Canthotomy,16637,133896,1,M,2081-11-07,57
1,851,Canthotomy,18875,189045,1,M,2098-12-20,74
2,851,Canthotomy,13516,191600,1,M,2062-10-25,38
3,851,Canthotomy,23407,171982,1,M,2152-09-11,128
4,851,Canthotomy,13376,166959,9,M,2124-04-09,100


In [36]:
#Convertir la colonne 'colonne_date' en type 'datetime'
Mimi_III_df['DOB'] = pd.to_datetime(Mimi_III_df['DOB'])


In [37]:
#Extraire la partie de la date et convertir en type 'date'
def from_dob_to_age(born):
    today =date.today()

    age=today.year - born.year - ((today.month, today.day) < (born.month, born.day))
    return age

In [38]:
age=Mimi_III_df['DOB'].apply(from_dob_to_age)
Mimi_III_df['age']=abs(age)

In [39]:
Age1=Mimi_III_df['ADMITTIME'].apply(from_dob_to_age)
Mimi_III_df['Age1']=abs(age)

KeyError: 'ADMITTIME'

In [41]:
#calcule age de pation
from datetime import date
def from_dob_to_age(born):
    today =date.today()

    Age1=today.year - born.year - ((today.month, today.day) < (born.month, born.day))
    return Age1
Age1=Mimi_III_df['ADMITTIME'].apply(from_dob_to_age)
Mimi_III_df['Age1']=abs(Age1)
pd.set_option('display.max_rows', None)
Mimi_III_df['Age2']=Mimi_III_df['Age1']-Mimi_III_df['age']
pd.set_option('display.max_rows', None)

KeyError: 'ADMITTIME'

In [23]:
# Liste des noms des colonnes à supprimer
colonnes_a_supprimer = ['DOB','age','Age1','ADMITTIME']
#Supprimer plusieurs colonnes simultanément
Mimi_III_df = Mimi_III_df.drop(columns=colonnes_a_supprimer)
Mimi_III_df = Mimi_III_df.rename(columns={'Age2': 'AGE PATIENT'})

In [42]:
#modifier les données non numériques GENDER
Mimi_III_df['GENDER'].replace(['M','F'],[1,0],inplace=True)

In [43]:
Mimi_III_df = Mimi_III_df.rename(columns={'ICD9_CODE': 'ICD-9- PROCEDURES'})

In [44]:
Mimi_III_df = Mimi_III_df.rename(columns={'LONG_TITLE_ICD-9- PROCEDURES': 'LONG_TITLE_PROCEDURES'})

In [45]:
diagnoses_icd_df=pd.read_csv('./data/DIAGNOSES_ICD.csv')

In [47]:
diagnoses_icd_df.shape

(651047, 5)

In [28]:
# Supprimer les lignes contenant des valeurs nulles
diagnoses_icd_df.dropna(axis=0, inplace=True)
diagnoses_icd_df.head()

,ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE
0,1297,109,172335,1.0,40301
1,1298,109,172335,2.0,486
2,1299,109,172335,3.0,58281
3,1300,109,172335,4.0,5855
4,1301,109,172335,5.0,4254


In [48]:
d_icd_diag=pd.read_csv('./data/D_ICD_DIAGNOSES.csv')

In [30]:
# Supprimer les lignes contenant des valeurs nulles
d_icd_diag.dropna(axis=0, inplace=True)
d_icd_diag.head()

,ROW_ID,ICD9_CODE,SHORT_TITLE,LONG_TITLE
0,174,01166,TB pneumonia-oth test,"Tuberculous pneumonia [any form], tubercle bac..."
1,175,01170,TB pneumothorax-unspec,"Tuberculous pneumothorax, unspecified"
2,176,01171,TB pneumothorax-no exam,"Tuberculous pneumothorax, bacteriological or h..."
3,177,01172,TB pneumothorx-exam unkn,"Tuberculous pneumothorax, bacteriological or h..."
4,178,01173,TB pneumothorax-micro dx,"Tuberculous pneumothorax, tubercle bacilli fou..."


In [49]:
# Fusionner les DataFrames'diagnoses_icd_df' et 'd_icd_diag' sur la colonne ICD9_CODE
merged_diagnos = pd.merge(diagnoses_icd_df, d_icd_diag, on=['ICD9_CODE', 'ICD9_CODE'], how='inner')

In [50]:
merged_diagnos = merged_diagnos.drop_duplicates(['SUBJECT_ID', 'HADM_ID'])


In [51]:
merged_diagnos.head(100)

,ROW_ID_x,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE,ROW_ID_y,SHORT_TITLE,LONG_TITLE
0,1297,109,172335,1.0,40301,4312,Mal hyp kid w cr kid V,"Hypertensive chronic kidney disease, malignant..."
14,1311,109,173633,1.0,40301,4312,Mal hyp kid w cr kid V,"Hypertensive chronic kidney disease, malignant..."
28,1488,112,174105,1.0,53100,6022,Ac stomach ulcer w hem,"Acute gastric ulcer with hemorrhage, without m..."
33,1493,113,109976,1.0,1915,2026,Mal neo cereb ventricle,Malignant neoplasm of ventricles
36,1496,114,178393,1.0,41401,4374,Crnry athrscl natve vssl,Coronary atherosclerosis of native coronary ar...
...,...,...,...,...,...,...,...,...
661,81,17,194023,1.0,7455,7314,Secundum atrial sept def,Ostium secundum type atrial septal defect
665,85,18,188822,1.0,25080,2271,DMII oth nt st uncntrld,"Diabetes with other specified manifestations, ..."
678,98,19,109235,1.0,80502,12438,Fx c2 vertebra-closed,Closed fracture of second cervical vertebra
687,107,20,157681,1.0,41401,4374,Crnry athrscl natve vssl,Coronary atherosclerosis of native coronary ar...


In [34]:
# Liste des noms des colonnes à supprimer
colonnes_a_supprimer = ['ROW_ID_x','ROW_ID_y','SHORT_TITLE','SEQ_NUM']
#Supprimer plusieurs colonnes simultanément
merged_diagnos = merged_diagnos.drop(columns=colonnes_a_supprimer)

In [52]:
merged_diagnos.head(100)

,ROW_ID_x,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE,ROW_ID_y,SHORT_TITLE,LONG_TITLE
0,1297,109,172335,1.0,40301,4312,Mal hyp kid w cr kid V,"Hypertensive chronic kidney disease, malignant..."
14,1311,109,173633,1.0,40301,4312,Mal hyp kid w cr kid V,"Hypertensive chronic kidney disease, malignant..."
28,1488,112,174105,1.0,53100,6022,Ac stomach ulcer w hem,"Acute gastric ulcer with hemorrhage, without m..."
33,1493,113,109976,1.0,1915,2026,Mal neo cereb ventricle,Malignant neoplasm of ventricles
36,1496,114,178393,1.0,41401,4374,Crnry athrscl natve vssl,Coronary atherosclerosis of native coronary ar...
...,...,...,...,...,...,...,...,...
661,81,17,194023,1.0,7455,7314,Secundum atrial sept def,Ostium secundum type atrial septal defect
665,85,18,188822,1.0,25080,2271,DMII oth nt st uncntrld,"Diabetes with other specified manifestations, ..."
678,98,19,109235,1.0,80502,12438,Fx c2 vertebra-closed,Closed fracture of second cervical vertebra
687,107,20,157681,1.0,41401,4374,Crnry athrscl natve vssl,Coronary atherosclerosis of native coronary ar...


In [53]:
Mimi_III_df = Mimi_III_df.rename(columns={'LONG_TITLE': 'LONG_TITLE_PROCEDURES'})

In [54]:
Mimi_III_df.head(100)

,ICD-9- PROCEDURES,LONG_TITLE_PROCEDURES,SUBJECT_ID,HADM_ID,SEQ_NUM,GENDER,DOB,age
0,851,Canthotomy,16637,133896,1,1,2081-11-07,57
1,851,Canthotomy,18875,189045,1,1,2098-12-20,74
2,851,Canthotomy,13516,191600,1,1,2062-10-25,38
3,851,Canthotomy,23407,171982,1,1,2152-09-11,128
4,851,Canthotomy,13376,166959,9,1,2124-04-09,100
...,...,...,...,...,...,...,...,...
95,863,Reconstruction of eyelid with hair follicle graft,55491,186490,2,0,2155-12-29,131
96,864,Reconstruction of eyelid with tarsoconjunctiva...,9873,149271,2,0,2089-05-23,65
97,864,Reconstruction of eyelid with tarsoconjunctiva...,12047,158491,1,1,2117-10-03,93
98,864,Reconstruction of eyelid with tarsoconjunctiva...,23829,158842,5,0,2105-03-31,81


In [55]:
# Fusionner les DataFrames'Mimi_III_df' et 'diagnoses_icd_df' sur la colonne SUBJECT_ID
Mimi_III_df = pd.merge(Mimi_III_df, merged_diagnos, on=['SUBJECT_ID', 'HADM_ID'], how='inner')

In [56]:
Mimi_III_df.head(100)

,ICD-9- PROCEDURES,LONG_TITLE_PROCEDURES,SUBJECT_ID,HADM_ID,SEQ_NUM_x,GENDER,DOB,age,ROW_ID_x,SEQ_NUM_y,ICD9_CODE,ROW_ID_y,SHORT_TITLE,LONG_TITLE
0,851,Canthotomy,16637,133896,1,1,2081-11-07,57,185121,1.0,80106,8286,Cl skul base fx-coma NOS,Closed fracture of base of skull without menti...
1,851,Canthotomy,18875,189045,1,1,2098-12-20,74,210018,1.0,80121,8297,Cl skul base fx w/o coma,Closed fracture of base of skull with subarach...
2,851,Canthotomy,13516,191600,1,1,2062-10-25,38,151549,1.0,9509,13380,Inj optic nerv/path NOS,Injury to unspecified optic nerve and pathways
3,851,Canthotomy,23407,171982,1,1,2152-09-11,128,260939,1.0,80125,7763,Cl skl base fx-deep coma,Closed fracture of base of skull with subarach...
4,851,Canthotomy,13376,166959,9,1,2124-04-09,100,150212,1.0,80126,7764,Cl skul base fx-coma NOS,Closed fracture of base of skull with subarach...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,863,Reconstruction of eyelid with hair follicle graft,55491,186490,2,0,2155-12-29,131,450229,1.0,23331,2556,Carcinoma in situ vagina,"Carcinoma in situ, vagina"
96,864,Reconstruction of eyelid with tarsoconjunctiva...,9873,149271,2,0,2089-05-23,65,110640,1.0,72886,13805,Necrotizing fasciitis,Necrotizing fasciitis
97,864,Reconstruction of eyelid with tarsoconjunctiva...,12047,158491,1,1,2117-10-03,93,135956,1.0,7078,8132,Chronic skin ulcer NEC,Chronic ulcer of other specified sites
98,864,Reconstruction of eyelid with tarsoconjunctiva...,23829,158842,5,0,2105-03-31,81,266272,1.0,25060,2263,DMII neuro nt st uncntrl,"Diabetes with neurological manifestations, typ..."


In [57]:
Mimi_III_df = Mimi_III_df.rename(columns={'LONG_TITLE': 'LONG_TITLE_DIAGNOSIS'})
Mimi_III_df = Mimi_III_df.rename(columns={'ICD9_CODE': 'CD9_CODE_DIAGNOSIS'})

In [60]:
Mimi_III_df.shape

(246174, 14)

In [58]:
prescriptions_df=pd.read_csv('./data/PRESCRIPTIONS.csv')

In [61]:
prescriptions_df = prescriptions_df.drop_duplicates(['SUBJECT_ID', 'HADM_ID'])


In [62]:
prescriptions_df.head(100)

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTDATE,ENDDATE,DRUG_TYPE,DRUG,DRUG_NAME_POE,DRUG_NAME_GENERIC,FORMULARY_DRUG_CD,GSN,NDC,PROD_STRENGTH,DOSE_VAL_RX,DOSE_UNIT_RX,FORM_VAL_DISP,FORM_UNIT_DISP,ROUTE
0,2214776,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,MAIN,Tacrolimus,Tacrolimus,Tacrolimus,TACR1,021796,4.690617e+08,1mg Capsule,2,mg,2,CAP,PO
23,2122384,13,143045,NaN,2167-01-08 00:00:00,2167-01-09 00:00:00,BASE,D5W,NaN,NaN,HEPBASE,NaN,0.000000e+00,HEPARIN BASE,250,ml,250,ml,IV
71,2378250,8,159514,NaN,2117-11-20 00:00:00,2117-11-24 00:00:00,BASE,Send 500mg Vial,NaN,NaN,AMPVL,NaN,0.000000e+00,Send 500mg Vial,1,VIAL,1,VIAL,IV
75,2088377,9,150750,NaN,2149-11-09 00:00:00,2149-11-09 00:00:00,BASE,SW,NaN,NaN,KCLBASE,NaN,0.000000e+00,100ml Bag,100,ml,100,ml,IV
107,2968761,2,163353,243653.0,2138-07-18 00:00:00,2138-07-20 00:00:00,MAIN,NEO*IV*Gentamicin,NaN,NaN,GENT10I,009298,6.332302e+10,10mg/mL-2mL,15.5,mg,0.775,VIAL,IV
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5888,1834936,109,136572,286860.0,2142-06-18 00:00:00,2142-06-20 00:00:00,MAIN,Heparin,Heparin,Heparin Sodium,HEPA5I,006549,6.332303e+10,5000 Units / mL- 1mL Vial,5000,UNIT,1,mL,SC
5898,1835660,109,174489,NaN,2142-07-03 00:00:00,2142-07-04 00:00:00,MAIN,PredniSONE,PredniSONE,PredniSONE,PRED1,006748,5.487392e+07,1 mg Tablet,4,mg,4,TAB,PO
5971,1837139,109,196721,214027.0,2142-07-15 00:00:00,2142-07-15 00:00:00,MAIN,Vancomycin,NaN,NaN,VANC500I,009331,4.094332e+08,500mg Vial,500,mg,1,VIAL,IV
5991,2078815,117,140784,211700.0,2133-04-10 00:00:00,2133-04-12 00:00:00,MAIN,Ipratropium Bromide Neb,Ipratropium Bromide Neb,Ipratropium Bromide Neb,IPRA2H,021700,4.879801e+08,2.5ml Vial,1,NEB,1,VIAL,IH


In [63]:
prescriptions_df=prescriptions_df[["SUBJECT_ID","HADM_ID","DRUG_TYPE","DRUG","PROD_STRENGTH","DOSE_VAL_RX","DOSE_UNIT_RX","ROUTE"]]

In [64]:
# Fusionner les DataFrames'Mimi_III_df' et 'prescriptions_df' sur la colonne SUBJECT_ID
Mimi_III_df = pd.merge(Mimi_III_df, prescriptions_df, on=['SUBJECT_ID', 'HADM_ID'], how='inner')

In [65]:
Mimi_III_df.head(100)

,ICD-9- PROCEDURES,LONG_TITLE_PROCEDURES,SUBJECT_ID,HADM_ID,SEQ_NUM_x,GENDER,DOB,age,ROW_ID_x,SEQ_NUM_y,CD9_CODE_DIAGNOSIS,ROW_ID_y,SHORT_TITLE,LONG_TITLE_DIAGNOSIS,DRUG_TYPE,DRUG,PROD_STRENGTH,DOSE_VAL_RX,DOSE_UNIT_RX,ROUTE
0,851,Canthotomy,18875,189045,1,1,2098-12-20,74,210018,1.0,80121,8297,Cl skul base fx w/o coma,Closed fracture of base of skull with subarach...,BASE,D5W,250ML BAG,250,ml,IV DRIP
1,851,Canthotomy,13516,191600,1,1,2062-10-25,38,151549,1.0,9509,13380,Inj optic nerv/path NOS,Injury to unspecified optic nerve and pathways,MAIN,Sodium Chloride 0.9% Flush,Syringe,3,ml,IV
2,851,Canthotomy,23407,171982,1,1,2152-09-11,128,260939,1.0,80125,7763,Cl skl base fx-deep coma,Closed fracture of base of skull with subarach...,MAIN,Humulin-R Insulin,10ML VIAL,100,UNIT,IV DRIP
3,851,Canthotomy,13376,166959,9,1,2124-04-09,100,150212,1.0,80126,7764,Cl skul base fx-coma NOS,Closed fracture of base of skull with subarach...,BASE,Soln.,50ml Bottle,50,ml,IV
4,851,Canthotomy,6282,178025,1,1,2069-10-16,45,70413,1.0,80120,8296,Cl skl base fx/menin hem,Closed fracture of base of skull with subarach...,MAIN,Haloperidol,5mg/mL Vial,2.5,mg,IM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,864,Reconstruction of eyelid with tarsoconjunctiva...,1354,152633,1,1,2055-03-04,30,15543,1.0,1725,1263,Malig melanoma trunk,"Malignant melanoma of skin of trunk, except sc...",MAIN,Allopurinol,300mg Tab,300,mg,PO
96,864,Reconstruction of eyelid with tarsoconjunctiva...,27665,186634,2,1,2113-03-10,88,312400,1.0,44024,4204,Ath ext ntv art gngrene,Atherosclerosis of native arteries of the extr...,MAIN,Furosemide,40mg Tablet,120,mg,PO
97,864,Reconstruction of eyelid with tarsoconjunctiva...,90540,183305,7,1,2087-10-09,63,607616,1.0,1710,1350,Mal neo soft tissue head,Malignant neoplasm of connective and other sof...,MAIN,Magnesium Sulfate,2 g / 50 mL Premix Bag,2,gm,IV
98,864,Reconstruction of eyelid with tarsoconjunctiva...,690,135389,1,1,2109-09-24,85,7936,1.0,03840,653,Gram-neg septicemia NOS,"Septicemia due to gram-negative organism, unsp...",BASE,D5W,150mL Bag,150,ml,IV


In [66]:
# Liste des noms des colonnes à supprimer
colonnes_a_supprimer = ['DISCHTIME','DEATHTIME']
#Supprimer plusieurs colonnes simultanément
Mimi_III_df = Mimi_III_df.drop(columns=colonnes_a_supprimer)

KeyError: "['DISCHTIME', 'DEATHTIME'] not found in axis"

In [84]:
Mimi_III_df.shape

(44405, 20)

In [49]:
MIMIC_III_subset = Mimi_III_df.head(500000)

In [78]:
Mimi_III_df.to_csv("Mimi_III_df.csv",index=False)

In [79]:
Mimi_III_df = Mimi_III_df.drop_duplicates(['SUBJECT_ID', 'HADM_ID'])


In [69]:
labevent_df=pd.read_csv('./data/LABEVENTS.csv')
# Supprimer les lignes contenant des valeurs nulles
labevent_df.dropna(axis=0, inplace=True)
labevent_df.shape

(7881769, 9)

In [70]:
labevent_df.head()

,ROW_ID,SUBJECT_ID,HADM_ID,ITEMID,CHARTTIME,VALUE,VALUENUM,VALUEUOM,FLAG
162,443,3,145834.0,50893,2101-10-20 16:40:00,8.2,8.2,mg/dL,abnormal
163,444,3,145834.0,50902,2101-10-20 16:40:00,99,99.0,mEq/L,abnormal
166,447,3,145834.0,50912,2101-10-20 16:40:00,3.2,3.2,mg/dL,abnormal
169,450,3,145834.0,50970,2101-10-20 16:40:00,4.8,4.8,mg/dL,abnormal
170,451,3,145834.0,50971,2101-10-20 16:40:00,5.4,5.4,mEq/L,abnormal


In [71]:
copie_labevent_df=pd.DataFrame(labevent_df)

In [54]:
# Liste des noms des colonnes à supprimer
colonnes_a_supprimer = ['ROW_ID']
#Supprimer plusieurs colonnes simultanément
copie_labevent_df = copie_labevent_df.drop(columns=colonnes_a_supprimer)

In [81]:
merged_diagnos = copie_labevent_df.drop_duplicates(['SUBJECT_ID', 'HADM_ID'])




In [85]:
# Sort and keep only one row per SUBJECT_ID and HADM_ID in copie_labevent_df
copie_labevent_df = copie_labevent_df.sort_values('CHARTTIME').drop_duplicates(['SUBJECT_ID', 'HADM_ID'], keep='first')
# Then merge
MIMIC_III_sub = pd.merge(copie_labevent_df, Mimi_III_df, on=['SUBJECT_ID', 'HADM_ID'], how='inner')

In [82]:
# Fusionner les DataFrames'copie_admissions_df' et 'merged_procedure' sur la colonne SUBJECT_ID
MIMIC_III_sub = pd.merge(copie_labevent_df, Mimi_III_df, on=['SUBJECT_ID', 'HADM_ID'], how='inner')

In [86]:
MIMIC_III_sub.shape

(44290, 27)

In [87]:
MIMIC_III_sub.to_csv("MIMIC_III_sub.csv",index=False)

In [142]:
MIMIC_III_sub=pd.read_csv('./MIMIC_III_sub.csv')


In [88]:
MIMIC_III_sub.shape

(44290, 27)

In [89]:
lab_not_event_df=pd.read_csv('./data/NOTEEVENTS-002.csv')


In [90]:
lab_not_event_df.shape

(2083180, 11)

In [61]:
# Remove rows with null values
lab_not_event_df = lab_not_event_df.dropna()

# Reset the index after removing rows
lab_not_event_df = lab_not_event_df.reset_index(drop=True)

In [130]:
lab_not_event_df.shape

(2083180, 11)

In [131]:
lab_not_event_df = lab_not_event_df.head(60000)


In [132]:
lab_not_event_df.shape


(60000, 11)

In [98]:
lab_not_event_df.head(10)

,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CATEGORY,DESCRIPTION,TEXT
0,174,22532,167853.0,2151-08-04,Discharge summary,Report,Admission Date: [**2151-7-16**] Dischar...
1,175,13702,107527.0,2118-06-14,Discharge summary,Report,Admission Date: [**2118-6-2**] Discharg...
2,176,13702,167118.0,2119-05-25,Discharge summary,Report,Admission Date: [**2119-5-4**] D...
3,177,13702,196489.0,2124-08-18,Discharge summary,Report,Admission Date: [**2124-7-21**] ...
4,178,26880,135453.0,2162-03-25,Discharge summary,Report,Admission Date: [**2162-3-3**] D...
5,179,53181,170490.0,2172-03-08,Discharge summary,Report,Admission Date: [**2172-3-5**] D...
6,180,20646,134727.0,2112-12-10,Discharge summary,Report,Admission Date: [**2112-12-8**] ...
7,181,42130,114236.0,2150-03-01,Discharge summary,Report,Admission Date: [**2150-2-25**] ...
8,182,56174,163469.0,2118-08-12,Discharge summary,Report,Admission Date: [**2118-8-10**] ...
9,183,56174,189681.0,2118-12-09,Discharge summary,Report,Admission Date: [**2118-12-7**] ...


In [99]:
# List of columns to remove
columns_to_drop = ['CHARTTIME', 'STORETIME', 'CGID', 'ISERROR']

# Remove the specified columns
lab_not_event_df = lab_not_event_df.drop(columns=columns_to_drop)

# Show the new dataframe info to confirm columns were removed
print("Updated DataFrame columns:")
print(lab_not_event_df.columns.tolist())

KeyError: "['CHARTTIME', 'STORETIME', 'CGID', 'ISERROR'] not found in axis"

In [100]:

lab_not_event_df = lab_not_event_df.drop_duplicates(['SUBJECT_ID', 'HADM_ID'])


In [101]:
lab_not_event_df.shape

(82178, 7)

In [102]:
lab_not_event_df.head(10)

,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CATEGORY,DESCRIPTION,TEXT
0,174,22532,167853.0,2151-08-04,Discharge summary,Report,Admission Date: [**2151-7-16**] Dischar...
1,175,13702,107527.0,2118-06-14,Discharge summary,Report,Admission Date: [**2118-6-2**] Discharg...
2,176,13702,167118.0,2119-05-25,Discharge summary,Report,Admission Date: [**2119-5-4**] D...
3,177,13702,196489.0,2124-08-18,Discharge summary,Report,Admission Date: [**2124-7-21**] ...
4,178,26880,135453.0,2162-03-25,Discharge summary,Report,Admission Date: [**2162-3-3**] D...
5,179,53181,170490.0,2172-03-08,Discharge summary,Report,Admission Date: [**2172-3-5**] D...
6,180,20646,134727.0,2112-12-10,Discharge summary,Report,Admission Date: [**2112-12-8**] ...
7,181,42130,114236.0,2150-03-01,Discharge summary,Report,Admission Date: [**2150-2-25**] ...
8,182,56174,163469.0,2118-08-12,Discharge summary,Report,Admission Date: [**2118-8-10**] ...
9,183,56174,189681.0,2118-12-09,Discharge summary,Report,Admission Date: [**2118-12-7**] ...


In [104]:
MIMIC_III_sub = pd.merge(
    lab_not_event_df, 
    MIMIC_III_sub, 
    on=['SUBJECT_ID', 'HADM_ID'], 
    how='inner', 
    suffixes=('_lab', '_mimic')  # Custom suffixes
)


In [105]:
MIMIC_III_sub.head(100)

,ROW_ID_lab,SUBJECT_ID,HADM_ID,CHARTDATE,CATEGORY,DESCRIPTION,TEXT,ROW_ID_mimic,ITEMID,CHARTTIME,VALUE,VALUENUM,VALUEUOM,FLAG,ICD-9- PROCEDURES,LONG_TITLE_PROCEDURES,SEQ_NUM_x,GENDER,DOB,age,ROW_ID_x,SEQ_NUM_y,CD9_CODE_DIAGNOSIS,ROW_ID_y,SHORT_TITLE,LONG_TITLE_DIAGNOSIS,DRUG_TYPE,DRUG,PROD_STRENGTH,DOSE_VAL_RX,DOSE_UNIT_RX,ROUTE
0,175,13702,107527.0,2118-06-14,Discharge summary,Report,Admission Date: [**2118-6-2**] Discharg...,8524304,51301,2118-06-02 07:30:00,19.0,19.00,K/uL,abnormal,3199,Other operations on trachea,1,0,2037-05-03,13,153653,2.0,49121,5545,Obs chr bronc w(ac) exac,Obstructive chronic bronchitis with (acute) ex...,MAIN,Pantoprazole,40mg Tab,40,mg,PO
1,176,13702,167118.0,2119-05-25,Discharge summary,Report,Admission Date: [**2119-5-4**] D...,8524896,51200,2119-05-04 22:21:00,5.7,5.70,%,abnormal,14,Injection or infusion of oxazolidinone class o...,8,0,2037-05-03,13,153661,3.0,496,5123,Chr airway obstruct NEC,"Chronic airway obstruction, not elsewhere clas...",BASE,Vial,Send Vial,1,VIAL,IV
2,177,13702,196489.0,2124-08-18,Discharge summary,Report,Admission Date: [**2124-7-21**] ...,8525989,50902,2124-07-21 09:55:00,92,92.00,mEq/L,abnormal,3891,Arterial catheterization,3,0,2037-05-03,13,153665,1.0,51884,5282,Acute & chronc resp fail,Acute and chronic respiratory failure,MAIN,PredniSONE,20 mg Tablet,40,mg,PO/NG
3,178,26880,135453.0,2162-03-25,Discharge summary,Report,Admission Date: [**2162-3-3**] D...,16630780,51279,2162-03-03 15:40:00,4.43,4.43,m/uL,abnormal,353,Repair of vertebral fracture,3,1,2080-01-04,55,301092,1.0,80506,12442,Fx c6 vertebra-closed,Closed fracture of sixth cervical vertebra,MAIN,CefazoLIN,1g Froz.Bag,1,g,IV
4,179,53181,170490.0,2172-03-08,Discharge summary,Report,Admission Date: [**2172-3-5**] D...,21694014,50824,2172-03-05 12:30:00,132,132.00,mEq/L,abnormal,34,Imageless computer assisted surgery,3,0,2109-10-08,85,439490,1.0,2252,2503,Ben neo cerebr meninges,Benign neoplasm of cerebral meninges,MAIN,CefazoLIN,10g Bulk vial,2,g,IV
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,57,1286,110814.0,2125-02-22,Discharge summary,Report,Admission Date: [**2125-2-13**] Dischar...,856304,51248,2125-02-12 23:45:00,32.9,32.90,pg,abnormal,8841,Arteriography of cerebral arteries,1,1,2062-05-02,38,14681,1.0,431,4503,Intracerebral hemorrhage,Intracerebral hemorrhage,MAIN,Pantoprazole,40MG TAB,40,mg,PO
96,58,1286,154417.0,2132-04-12,Discharge summary,Report,Admission Date: [**2132-4-10**] ...,856529,50931,2132-04-10 10:28:00,153,153.00,mg/dL,abnormal,4513,Other endoscopy of small intestine,1,1,2062-05-02,38,14685,1.0,5780,5866,Hematemesis,Hematemesis,MAIN,Erythromycin,500 mg Vial,250,mg,IV
97,59,5771,127184.0,2173-07-15,Discharge summary,Report,Admission Date: [**2173-6-30**] ...,3632505,50912,2173-06-30 07:40:00,8.7,8.70,mg/dL,abnormal,3949,Other revision of vascular procedure,5,1,2095-06-20,71,64826,1.0,5602,5585,Volvulus of intestine,Volvulus,MAIN,HYDROmorphone (Dilaudid),12.5mg/50mL Syringe,12.5,mg,IVPCA
98,60,5771,128201.0,2173-07-28,Discharge summary,Report,Admission Date: [**2173-7-24**] ...,3633139,50912,2173-07-24 06:00:00,4.6,4.60,mg/dL,abnormal,3995,Hemodialysis,1,1,2095-06-20,71,64840,1.0,56212,4946,Dvrtclo colon w hmrhg,Diverticulosis of colon with hemorrhage,MAIN,Insulin,Dummy Package for Sliding Scale,0,UNIT,SC


In [107]:
MIMIC_III_sub.to_csv("MIMIC_III_finale.csv",index=False)

In [106]:
MIMIC_III_sub.shape

(44048, 32)

In [110]:
import pandas as pd
import time
import openai
import os
from dotenv import load_dotenv
load_dotenv()
# Load dataset
df = pd.read_csv("MIMIC_III_finale.csv")
df = df.head(10000)

# Initialize OpenAI client
client = openai.OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

def extract_medical_info(text):
    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "Extract 'Maladie chronique', 'symptômes', 'allergies' and 'traitement régulier' from the following medical text."},
                {"role": "user", "content": text}
            ],
            temperature=0.1
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"Error processing text: {e}")
        return None

def parse_medical_info(parts):
    chronic, sympt, allergies, treatment = [], [], [], []
    current_section = None

    for part in parts:
        part = part.strip()
        if not part:
            continue

        if part.startswith("- Maladie chronique"):
            current_section = chronic
        elif part.startswith("- Symptômes"):
            current_section = sympt
        elif part.startswith("- Allergies"):
            current_section = allergies
        elif part.startswith("- Traitement régulier"):
            current_section = treatment
        elif current_section is not None:
            current_section.append(part)

    return (
        "; ".join(chronic),  
        "; ".join(sympt),  
        "; ".join(allergies),  
        "; ".join(treatment)  
    )


# Initialize columns for extracted data
df["Maladie_chronique"] = ""
df["Symptômes"] = ""
df["Allergies"] = ""
df["Traitement_régulier"] = ""

# Process in batches of 10
batch_size = 10
total_rows = len(df)

for start_idx in range(0, total_rows, batch_size):
    end_idx = min(start_idx + batch_size, total_rows)
    print(f"Processing rows {start_idx} to {end_idx-1}...")
    
    # Process batch
    for index in range(start_idx, end_idx):
        text = df.iloc[index]["TEXT"]
        extracted_info = extract_medical_info(text)
        
        if extracted_info:
            parts = extracted_info.split("\n")
            print(f"Processing row {index}:")
            chronic, sympt, allergies, treatment = parse_medical_info(parts)
            
            # Debug print
            print(f"Extracted Maladie chronique: {chronic[:50]}...")
            print(f"Extracted Symptômes: {sympt[:50]}...")
            print(f"Extracted Allergies: {allergies[:50]}...")
            print(f"Extracted Traitement régulier: {treatment[:50]}...")
            print("-" * 50)
            
            df.at[index, "Maladie_chronique"] = chronic
            df.at[index, "Symptômes"] = sympt
            df.at[index, "Allergies"] = allergies
            df.at[index, "Traitement_régulier"] = treatment
        
        time.sleep(1)  # Rate limiting
    
    # Save after each batch
    temp_filename = f"mimic3_dataset_updated_batch_{start_idx//batch_size}.csv"
    df.to_csv(temp_filename, index=False)
    print(f"Batch {start_idx//batch_size} completed and saved to {temp_filename}")

# Save final complete dataset
df.to_csv("mimic3_dataset_updated_final.csv", index=False)
print("Processing complete. Final dataset saved to mimic3_dataset_updated_final.csv")


Processing rows 0 to 9...
Processing row 0:
Extracted Maladie chronique: 1. Emphysema (COPD); 2. Lacunar CVA; 3. Angina; 4....
Extracted Symptômes: 1. Shortness of breath; 2. Chest pressure occasion...
Extracted Allergies: Norvasc leads to lightheadedness and headache...
Extracted Traitement régulier: 1. Hydrochlorothiazide; 2. Prednisone; 3. Levoflox...
--------------------------------------------------
Processing row 1:
Extracted Maladie chronique: ...
Extracted Symptômes: ...
Extracted Allergies: ...
Extracted Traitement régulier: ...
--------------------------------------------------
Processing row 2:
Extracted Maladie chronique: ...
Extracted Symptômes: ...
Extracted Allergies: ...
Extracted Traitement régulier: ...
--------------------------------------------------
Processing row 3:
Extracted Maladie chronique: ...
Extracted Symptômes: ...
Extracted Allergies: ...
Extracted Traitement régulier: ...
--------------------------------------------------
Processing row 4:
Extracted Mal

KeyboardInterrupt: 

In [1]:
import pandas as pd
import time
import openai
import os
from dotenv import load_dotenv
load_dotenv()
# Load dataset
df = pd.read_csv("MIMIC_III_finale_2.csv")
df = df.iloc[10000:20000]
# Process first 10,000 rows for testing

# Create output directory if it doesn't exist
output_dir = "mimic3_batches"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Initialize OpenAI client
client = openai.OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

def extract_medical_info(text):
    """ Calls OpenAI to extract medical details from text """
    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "Extract 'Maladie chronique', 'symptômes', 'allergies' and 'traitement régulier' from the following medical text."},
                {"role": "user", "content": text}
            ],
            temperature=0.1
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"Error processing text: {e}")
        return None

def parse_medical_info(parts):
    """ Parses extracted medical text into structured data """
    chronic, sympt, allergies, treatment = "Unknown", "Unknown", "Unknown", "Unknown"
    current_section = None
    for part in parts:
        part = part.strip()
        if not part:
            continue
        if "Maladie chronique" in part:
            current_section = "chronic"
            chronic = part.replace("- Maladie chronique:", "").strip()
        elif "Symptômes" in part:
            current_section = "sympt"
            sympt = part.replace("- Symptômes:", "").strip()
        elif "Allergies" in part:
            current_section = "allergies"
            allergies = part.replace("- Allergies:", "").strip()
        elif "Traitement régulier" in part:
            current_section = "treatment"
            treatment = part.replace("- Traitement régulier:", "").strip()
        elif current_section == "chronic":
            chronic += "; " + part
        elif current_section == "sympt":
            sympt += "; " + part
        elif current_section == "allergies":
            allergies += "; " + part
        elif current_section == "treatment":
            treatment += "; " + part
    return chronic, sympt, allergies, treatment

# Initialize new columns in DataFrame
df["Maladie_chronique"] = ""
df["Symptômes"] = ""
df["Allergies"] = ""
df["Traitement_régulier"] = ""

# Process data in batches
batch_size = 10
total_rows = len(df)

for start_idx in range(0, total_rows, batch_size):
    end_idx = min(start_idx + batch_size, total_rows)
    print(f"Processing rows {start_idx} to {end_idx-1}...")
    
    # Get the current batch
    batch_df = df.iloc[start_idx:end_idx].copy()
    
    # Process each row in the batch
    for i, (_, row) in enumerate(batch_df.iterrows()):
        idx = start_idx + i
        text = row["TEXT"]
        extracted_info = extract_medical_info(text)
        
        if extracted_info:
            parts = extracted_info.split("\n")
            chronic, sympt, allergies, treatment = parse_medical_info(parts)
            
            # Assign extracted data to batch DataFrame
            batch_df.at[batch_df.index[i], "Maladie_chronique"] = chronic
            batch_df.at[batch_df.index[i], "Symptômes"] = sympt
            batch_df.at[batch_df.index[i], "Allergies"] = allergies
            batch_df.at[batch_df.index[i], "Traitement_régulier"] = treatment
            
            # Also update the main DataFrame for completeness
            df.at[idx, "Maladie_chronique"] = chronic
            df.at[idx, "Symptômes"] = sympt
            df.at[idx, "Allergies"] = allergies
            df.at[idx, "Traitement_régulier"] = treatment
    
    # Remove TEXT column from the batch before saving
    batch_df = batch_df.drop(columns=["TEXT"])
    
    # Save this batch to a separate CSV file
    batch_filename = f"{output_dir}/mimic3_batch_{start_idx//batch_size + 1}.csv"
    batch_df.to_csv(batch_filename, index=False)
    
    print(f"✅ Batch {start_idx//batch_size + 1} processed. Saved to {batch_filename}")
    
    # Optional: add a small delay to avoid API rate limits
    time.sleep(0.5)

# Save the complete dataset without the TEXT column as well
df_final = df.drop(columns=["TEXT"])
df_final.to_csv("mimic3_dataset_updated_final_no_text.csv", index=False)

print("✅ Processing complete. All batches saved!")

# Debugging Check - Print info about saved files
print(f"Total batches created: {(total_rows + batch_size - 1) // batch_size}")
print(f"Files saved in directory: {output_dir}")
print(f"Final dataset (without TEXT) saved as: mimic3_dataset_updated_final_no_text.csv")

# Verify one batch file
first_batch = pd.read_csv(f"{output_dir}/mimic3_batch_1.csv")
print("\nFirst batch preview (should not contain TEXT column):")
print(first_batch.head(3))

Processing rows 0 to 9...
✅ Batch 1 processed. Saved to mimic3_batches/mimic3_batch_1.csv
Processing rows 10 to 19...
✅ Batch 2 processed. Saved to mimic3_batches/mimic3_batch_2.csv
Processing rows 20 to 29...
✅ Batch 3 processed. Saved to mimic3_batches/mimic3_batch_3.csv
Processing rows 30 to 39...
✅ Batch 4 processed. Saved to mimic3_batches/mimic3_batch_4.csv
Processing rows 40 to 49...
✅ Batch 5 processed. Saved to mimic3_batches/mimic3_batch_5.csv
Processing rows 50 to 59...
✅ Batch 6 processed. Saved to mimic3_batches/mimic3_batch_6.csv
Processing rows 60 to 69...
✅ Batch 7 processed. Saved to mimic3_batches/mimic3_batch_7.csv
Processing rows 70 to 79...
✅ Batch 8 processed. Saved to mimic3_batches/mimic3_batch_8.csv
Processing rows 80 to 89...
✅ Batch 9 processed. Saved to mimic3_batches/mimic3_batch_9.csv
Processing rows 90 to 99...
✅ Batch 10 processed. Saved to mimic3_batches/mimic3_batch_10.csv
Processing rows 100 to 109...
✅ Batch 11 processed. Saved to mimic3_batches/mimi

KeyboardInterrupt: 